# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f261401bb50>
├── 'a' --> tensor([[ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796]])
└── 'x' --> <FastTreeValue 0x7f261401b760>
    └── 'c' --> tensor([[-0.0831, -0.2791,  0.7694, -1.1180],
                        [ 0.2136, -0.6164, -1.3785, -0.5023],
                        [-1.2682, -0.0896,  0.3914,  0.1475]])

In [4]:
t.a

tensor([[ 0.2304,  0.7922,  1.4427],
        [-0.9715,  0.2259,  1.2796]])

In [5]:
%timeit t.a

74 ns ± 1.56 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f261401bb50>
├── 'a' --> tensor([[-0.4103,  0.2274, -1.5223],
│                   [-0.3212, -1.3128,  0.7879]])
└── 'x' --> <FastTreeValue 0x7f261401b760>
    └── 'c' --> tensor([[-0.0831, -0.2791,  0.7694, -1.1180],
                        [ 0.2136, -0.6164, -1.3785, -0.5023],
                        [-1.2682, -0.0896,  0.3914,  0.1475]])

In [7]:
%timeit t.a = new_value

77.3 ns ± 1.64 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796]]),
    x: Batch(
           c: tensor([[-0.0831, -0.2791,  0.7694, -1.1180],
                      [ 0.2136, -0.6164, -1.3785, -0.5023],
                      [-1.2682, -0.0896,  0.3914,  0.1475]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2304,  0.7922,  1.4427],
        [-0.9715,  0.2259,  1.2796]])

In [11]:
%timeit b.a

62.2 ns ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4819, -0.1286,  0.7698],
               [-0.2364,  1.2567,  0.1706]]),
    x: Batch(
           c: tensor([[-0.0831, -0.2791,  0.7694, -1.1180],
                      [ 0.2136, -0.6164, -1.3785, -0.5023],
                      [-1.2682, -0.0896,  0.3914,  0.1475]]),
       ),
)

In [13]:
%timeit b.a = new_value

538 ns ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

901 ns ± 22.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

253 µs ± 7.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

241 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f261407bd00>
├── 'a' --> tensor([[[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]],
│           
│                   [[ 0.2304,  0.7922,  1.4427],
│                    [-0.9715,  0.2259,  1.2796]]])
└── 'x' --> <FastTreeValue 0x7f25e87b0ac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.8 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f261408f190>
├── 'a' --> tensor([[ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796],
│                   [ 0.2304,  0.7922,  1.4427],
│                   [-0.9715,  0.2259,  1.2796]])
└── 'x' --> <FastTreeValue 0x7f2566cd0f40>
    └── 'c' --> tensor([[-0.0831, -0.2791,  0.7694, -1.1180],
                        [ 0.2136, -0.6164, -1.3785, -0.5023],
                 

In [23]:
%timeit t_cat(trees)

37.5 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.9 µs ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]],
       
               [[ 0.2304,  0.7922,  1.4427],
                [-0.9715,  0.2259,  1.2796]]]),
    x: Batch(
           c: tensor([[[-0.0831, -0.2791,  0.7694, -1.1180],
                       [ 0.2136, -0.6164, -1.3785, -0.5023],
                       [-1.2682, -0.0896,  0.3914,  0.1475]],
         

In [26]:
%timeit Batch.stack(batches)

90.4 µs ± 2.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796],
               [ 0.2304,  0.7922,  1.4427],
               [-0.9715,  0.2259,  1.2796]]),
    x: Batch(
           c: tensor([[-0.0831, -0.2791,  0.7694, -1.1180],
                      [ 0.2136, -0.6164, -1.3785, -0.5023],
                      [-1.2682, -0.0896,  0.3914,  0.1475],
                      [-0.0831, -0.2791,  0.7694, -1.1180],
                      [ 0.2136, -

In [28]:
%timeit Batch.cat(batches)

167 µs ± 4.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

497 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
